Importing stuff

In [ ]:
import numpy as np
import cv2
import h5py
import tifffile as tiff

# input filename
filename = 'z_030_div1.tif'
# z location for anatomy
za = 99

# reading filename
a = tiff.imread(filename)

# Extracting a single frame
extract = 82

zdata = a[extract]

# filesavname=filename[:-4]+'_single.tif'
# tiff.imsave(filesavname,filesav)

In [ ]:
# reading anatomical data from hdf5 file

anatomy = h5py.File('AnatomyLabelDatabase.hdf5','r')
anatomyGCaMP5 = anatomy['Elavl3-GCaMP5G_6dpf_MeanImageOf7Fish']
# desired z plane
zanatomy = anatomyGCaMP5[:][:][za]
cv2.imwrite('zanatomy.jpg',zanatomy)
print(zanatomy)

In [ ]:
# image registration
# img_data = cv2.cvtColor(zdata, cv2.COLOR_BGR2GRAY) 
# img_anat = cv2.cvtColor(zanatomy, cv2.COLOR_BGR2GRAY) 
[height, width] = zanatomy.shape 

# SIFT detector
orb = cv2.ORB()

# finding keypoints
kp1, d1 = orb.detectAndCompute(zdata, None) 
kp2, d2 = orb.detectAndCompute(zanatomy, None)

# creating a matcher
# using brute force hamming distance
matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck = True)

# matching descriptors
matches = matcher.match(d1, d2)
print(matches)
# Define empty matrices of shape no_of_matches * 2. 
p1 = np.zeros((no_of_matches, 2)) 
p2 = np.zeros((no_of_matches, 2)) 
  
for i in range(len(matches)): 
  p1[i, :] = kp1[matches[i].queryIdx].pt 
  p2[i, :] = kp2[matches[i].trainIdx].pt 

print(p1)
print(p2)
# Find the homography matrix. 
homography, mask = cv2.findHomography(p1, p2, cv2.RANSAC) 
print(homography)
# Use this matrix to transform the 
# colored image wrt the reference image. 
transformed_img = cv2.warpPerspective(zdata, 
                    homography, (width, height)) 
  
# Save the output. 
tiff.imsave('transformed.tif',transformed_img)
tiff.imsave('anatomy.tif',zanatomy)